In [6]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time
import re

# Set your Spotify API credentials
client_id = 'da07024f200a4ea99701afb3cf8e4054'
client_secret = '50d840c956114871af57aa7cd9341c2f'

# Initialize Spotify client
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Function to extract Spotify track ID from the URL or URI
def extract_track_id(spotify_url):
    if pd.isna(spotify_url):
        return None
    match = re.search(r'spotify:track:([a-zA-Z0-9]+)', spotify_url)
    if match:
        return match.group(1)
    return None

# Function to get audio features for a track
def get_audio_features(track_id):
    try:
        if track_id is None:
            return None
        
        features = sp.audio_features(track_id)[0]
        if features is None:
            return None
            
        return {
            'danceability': features['danceability'],
            'energy': features['energy'],
            'key': features['key'],
            'loudness': features['loudness'],
            'mode': features['mode'],
            'speechiness': features['speechiness'],
            'acousticness': features['acousticness'],
            'instrumentalness': features['instrumentalness'],
            'liveness': features['liveness'],
            'valence': features['valence'],
            'tempo': features['tempo'],  # This is beats per minute (BPM)
            'duration_ms': features['duration_ms']
        }
    except Exception as e:
        print(f"Error fetching features for track {track_id}: {e}")
        return None

# Load your CSV file
# Based on your example, I'll try to determine the structure, but you may need to adjust this
df = pd.read_csv('/Users/rajatsharma/Documents/introspectAI/introspect.ai/combining-apple-spotify/final-dataset-apple-spotify/spotifyHeartRate.csv')

# Handle the unusual format of your data
# You might need to adjust this based on the actual structure of your CSV
if len(df.columns) == 1:
    # If all data is in a single column, we need to parse it
    # The column might have a complex name that includes all the headers
    column_name = df.columns[0]
    
    # Extract the URL part from each row
    df['url'] = df[column_name].str.extract(r'(spotify:track:[a-zA-Z0-9]+)')

# Extract track IDs
df['track_id'] = df['url'].apply(extract_track_id)

# Get audio features for each track
features_list = []
batch_size = 50  # Process in batches to avoid rate limiting

for i in range(0, len(df), batch_size):
    batch = df.iloc[i:i+batch_size]
    print(f"Processing batch {i//batch_size + 1} of {(len(df) + batch_size - 1)//batch_size}")
    
    for idx, row in batch.iterrows():
        track_id = row['track_id']
        features = get_audio_features(track_id)
        if features:
            features['track_id'] = track_id
            features['original_index'] = idx
            features_list.append(features)
    
    # Sleep to avoid hitting rate limits
    if i + batch_size < len(df):
        time.sleep(1)

# Create a dataframe with the audio features
features_df = pd.DataFrame(features_list)

# Merge with the original dataframe
result_df = pd.merge(df, features_df, on='track_id', how='left')

# Save the results
result_df.to_csv('spotify_tracks_with_features.csv', index=False)

print(f"Analysis complete. Processed {len(features_list)} tracks successfully.")

HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=3HFjh7QljnUCBLsoTwMuTj with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=56zZ48jdyY2oDXHVnwg5Di with Params: {} returned 403 due to None


Processing batch 1 of 29
Error fetching features for track 3HFjh7QljnUCBLsoTwMuTj: http status: 403, code:-1 - https://api.spotify.com/v1/audio-features/?ids=3HFjh7QljnUCBLsoTwMuTj:
 None, reason: None
Error fetching features for track 56zZ48jdyY2oDXHVnwg5Di: http status: 403, code:-1 - https://api.spotify.com/v1/audio-features/?ids=56zZ48jdyY2oDXHVnwg5Di:
 None, reason: None


HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=5PvwPy5eRO8BPwpRzCHK3D with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=0WNCYgCd33LnBnw22WFuq8 with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=0d7cAnv2jF9H0pqNhovl7f with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=5fwRhPXAvGgzUxW62ZCvEg with Params: {} returned 403 due to None


Error fetching features for track 5PvwPy5eRO8BPwpRzCHK3D: http status: 403, code:-1 - https://api.spotify.com/v1/audio-features/?ids=5PvwPy5eRO8BPwpRzCHK3D:
 None, reason: None
Error fetching features for track 0WNCYgCd33LnBnw22WFuq8: http status: 403, code:-1 - https://api.spotify.com/v1/audio-features/?ids=0WNCYgCd33LnBnw22WFuq8:
 None, reason: None
Error fetching features for track 0d7cAnv2jF9H0pqNhovl7f: http status: 403, code:-1 - https://api.spotify.com/v1/audio-features/?ids=0d7cAnv2jF9H0pqNhovl7f:
 None, reason: None
Error fetching features for track 5fwRhPXAvGgzUxW62ZCvEg: http status: 403, code:-1 - https://api.spotify.com/v1/audio-features/?ids=5fwRhPXAvGgzUxW62ZCvEg:
 None, reason: None


HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=7JGgKHHDgJCJkQCQxyHHdl with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=4wIWWEhbdgKAxhR68eE0Ye with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=5fwRhPXAvGgzUxW62ZCvEg with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=1O5mTSQa0sCce9ghhDqIZl with Params: {} returned 403 due to None


...

HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=3rsx3pDYM3a08O0Yu3Tq6V with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=5CXKZm1KE7RGym7insLJM2 with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=3TAhWtQnpoL5Vl9VQPl9fU with Params: {} returned 403 due to None


Error fetching features for track 0xNc6Z322CXrfK0tAvOXpm: http status: 403, code:-1 - https://api.spotify.com/v1/audio-features/?ids=0xNc6Z322CXrfK0tAvOXpm:
 None, reason: None
Error fetching features for track 3rsx3pDYM3a08O0Yu3Tq6V: http status: 403, code:-1 - https://api.spotify.com/v1/audio-features/?ids=3rsx3pDYM3a08O0Yu3Tq6V:
 None, reason: None
Error fetching features for track 5CXKZm1KE7RGym7insLJM2: http status: 403, code:-1 - https://api.spotify.com/v1/audio-features/?ids=5CXKZm1KE7RGym7insLJM2:
 None, reason: None
Error fetching features for track 3TAhWtQnpoL5Vl9VQPl9fU: http status: 403, code:-1 - https://api.spotify.com/v1/audio-features/?ids=3TAhWtQnpoL5Vl9VQPl9fU:
 None, reason: None


HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=11ipwX6rA4f5KMTYB7QFXj with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=0biCSADTAblvLTLtJz4pXO with Params: {} returned 403 due to None


Error fetching features for track 11ipwX6rA4f5KMTYB7QFXj: http status: 403, code:-1 - https://api.spotify.com/v1/audio-features/?ids=11ipwX6rA4f5KMTYB7QFXj:
 None, reason: None
Error fetching features for track 0biCSADTAblvLTLtJz4pXO: http status: 403, code:-1 - https://api.spotify.com/v1/audio-features/?ids=0biCSADTAblvLTLtJz4pXO:
 None, reason: None


KeyError: 'track_id'

In [4]:
df.head()

,startDate,ms_played,track_name,artist_name,album_name,url,type,sourceName,value,unit,endDate
0,2023-11-09 18:30:00,0,Kya Mujhe Pyar Hai,KK,Woh Lamhe,spotify:track:3HFjh7QljnUCBLsoTwMuTj,HeartRate,Casio G-Shock,66.0,count/min,11/9/23 18:36
1,2023-11-09 18:40:00,115970,Tum Hi Ho,Mithoon,Aashiqui 2,spotify:track:56zZ48jdyY2oDXHVnwg5Di,HeartRate,Casio G-Shock,67.0,count/min,11/9/23 18:49
2,2023-11-09 18:50:00,390222,Sunn Raha Hai (Male Version),Ankit Tiwari,Aashiqui 2,spotify:track:5PvwPy5eRO8BPwpRzCHK3D,HeartRate,Casio G-Shock,61.0,count/min,11/9/23 18:55
3,2023-11-09 19:00:00,143149,Bhula Dena,Mustafa Zahid,Aashiqui 2,spotify:track:0WNCYgCd33LnBnw22WFuq8,HeartRate,Casio G-Shock,62.0,count/min,11/9/23 19:07
4,2023-11-09 19:10:00,1190,Tum Hi Ho (Remix),Mithoon,Aashiqui 2,spotify:track:0d7cAnv2jF9H0pqNhovl7f,HeartRate,Casio G-Shock,65.0,count/min,11/9/23 19:16
